In [15]:
# Import required libraries
from google.colab import drive
import os
import zipfile
import shutil
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [16]:
# Step 1: Connect to Google Drive
# drive.mount('/content/drive')

# Define file paths
zip_file_location = '/content/cifake_dataset.zip'  # Path to the CIFAKE dataset zip file
unzip_location = '/content'  # Destination folder for extracted files

# Step 2: Extract the ZIP file (may take some time)
with zipfile.ZipFile(zip_file_location, 'r') as zip_file:
    zip_file.extractall(unzip_location)

In [17]:
# Step 3: Set up paths for training and validation folders
real_train_path = os.path.join(unzip_location, 'cifake_dataset', 'train', 'REAL')
fake_train_path = os.path.join(unzip_location, 'cifake_dataset', 'train', 'FAKE')
validation_folder_path = os.path.join(unzip_location, 'cifake_dataset', 'validation')
real_validation_path = os.path.join(validation_folder_path, 'REAL')
fake_validation_path = os.path.join(validation_folder_path, 'FAKE')

# Create validation folders if they do not exist
os.makedirs(real_validation_path, exist_ok=True)
os.makedirs(fake_validation_path, exist_ok=True)

In [18]:
# Step 4: Transfer 10,000 images from train/REAL to validation/REAL and from train/FAKE to validation/FAKE
def transfer_images(source_folder, destination_folder, count):
    image_list = os.listdir(source_folder)
    random.shuffle(image_list)  # Shuffle images for random selection
    for image in image_list[:count]:
        source_image_path = os.path.join(source_folder, image)
        destination_image_path = os.path.join(destination_folder, image)
        shutil.move(source_image_path, destination_image_path)

# Move images
transfer_images(real_train_path, real_validation_path, 10000)
transfer_images(fake_train_path, fake_validation_path, 10000)

# **PART 2**

In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load datasets from the directories
train_ds = image_dataset_from_directory(
    os.path.join(unzip_location, 'cifake_dataset', 'train'),
    image_size=(32, 32),
    batch_size=32,
)

val_ds = image_dataset_from_directory(
    os.path.join(unzip_location, 'cifake_dataset', 'validation'),
    image_size=(32, 32),
    batch_size=32,
)

test_ds = image_dataset_from_directory(
    os.path.join(unzip_location, 'cifake_dataset', 'test'),
    image_size=(32, 32),
    batch_size=32,
)


Found 80000 files belonging to 2 classes.
Found 35989 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the enhanced CNN model with Batch Normalization and more layers
model = models.Sequential([
    # Rescaling layer
    layers.Rescaling(1.0 / 255, input_shape=(32, 32, 3)),

    # First Conv2D layer with ReLU activation, Batch Normalization, and Max Pooling
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Second Conv2D layer with increased filters, Batch Normalization, and Max Pooling
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Third Conv2D layer for deeper features, Batch Normalization, and Max Pooling
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Flatten the output for the dense layers
    layers.Flatten(),

    # First Dense layer with ReLU activation and Dropout
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    # Second Dense layer with ReLU activation and Dropout
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    # Output layer for binary classification
    layers.Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [26]:
# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define ModelCheckpoint callback to save the best model
checkpoint_cb = ModelCheckpoint("best_custom_model.keras", save_best_only=True, monitor="val_loss")

# Train the model for 30 epochs
history = model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=[checkpoint_cb])

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 30s 8ms/step - accuracy: 0.8464 - loss: 0.3709 - val_accuracy: 0.9175 - val_loss: 0.2059
Epoch 2/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 34s 8ms/step - accuracy: 0.9202 - loss: 0.2051 - val_accuracy: 0.9419 - val_loss: 0.1556
Epoch 3/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9288 - loss: 0.1824 - val_accuracy: 0.9493 - val_loss: 0.1365
Epoch 4/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9396 - loss: 0.1591 - val_accuracy: 0.9495 - val_loss: 0.1288
Epoch 5/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.9461 - loss: 0.1442 - val_accuracy: 0.8863 - val_loss: 0.3071
Epoch 6/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9530 - loss: 0.1242 - val_accuracy: 0.9457 - val_loss: 0.1401
Epoch 7/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.9599 - loss: 0.1099 - val_accuracy: 0.9622 - val_loss: 0.1009
Epoch 8/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9657 - loss: 0

In [27]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9578 - loss: 0.1640
Test Accuracy: 95.76%
